In [37]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import random

In [2]:
disc_inp_data = pd.read_csv('../Fresh_start/did_MIMIC_mimiciii_with_rewards.csv')

In [3]:
#Import the list of pregnant patients ICUstay IDs
pregnant_df = pd.read_csv("../Created Data/Pregnant_ids.csv")

In [13]:
pregnant_ids = pregnant_df["icustay_id"]-200000

In [14]:
pregnant_set = DataFrame()
pregnant_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(pregnant_ids)]

In [21]:
#List the age, sex and icustayid for each pregnant sepsis patient
icustayid = []
age=[]
sex=[]
pregnant_set = pregnant_set.reset_index()
for i, row in pregnant_set.iterrows():
    #print(i)
    if i == 0:
        j=i
        icustayid.append(pregnant_set.loc[i,"icustayid"])
        sex.append(pregnant_set.loc[i,"gender"])
        age.append(pregnant_set.loc[i,"age"])
        continue
    else:
        #print(row["bloc"])
        if pregnant_set.loc[i,"icustayid"] != pregnant_set.loc[j,"icustayid"]:
            icustayid.append(pregnant_set.loc[i,"icustayid"])
            sex.append(pregnant_set.loc[i,"gender"])
            age.append(pregnant_set.loc[i,"age"])
        j=i


In [23]:
#Turn the age, sex, icustayids lists into a dataframe
age_sex_preg_sep_df = pd.DataFrame(columns = ["icustayid", "age", "sex"])
age_sex_preg_sep_df["icustayid"] = icustayid
age_sex_preg_sep_df["age"] = age
age_sex_preg_sep_df["sex"] = sex

In [24]:
female_set = disc_inp_data.loc[disc_inp_data['gender'].isin([1])]

In [25]:
female_set.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,input_4hourly,output_total,output_4hourly,cumulated_balance,SOFA,SIRS,vaso_input,iv_input,action,reward
7,1,11,6898241400,1,30766.069,6,1,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,11,0,0.0,0.0,0,0.383136
8,2,11,6898255800,1,30766.069,6,1,NaN,NaN,0.0,...,0.0,460.0,460.0,-460.0,9,0,0.0,0.0,0,-0.271041
9,3,11,6898270200,1,30766.069,6,1,NaN,NaN,0.0,...,0.0,1020.0,560.0,-1020.0,7,1,0.0,0.0,0,-0.884898
10,4,11,6898284600,1,30766.069,6,1,NaN,NaN,0.0,...,0.0,1600.0,580.0,-1600.0,8,1,0.0,0.0,0,-0.025000
11,5,11,6898299000,1,30766.069,6,1,NaN,NaN,0.0,...,0.0,2020.0,420.0,-2020.0,8,1,0.0,0.0,0,-0.025000


In [26]:
female_not_preg_set = female_set.loc[~female_set['icustayid'].isin(pregnant_ids)]

In [28]:
not_preg_set = disc_inp_data.loc[~disc_inp_data['icustayid'].isin(pregnant_ids)]

In [29]:
#Find the age, sex and icustayid for each patient in the young_notpreg_df dataframe
y_icustayid = []
y_age=[]
y_sex=[]
for i, row in female_not_preg_set.iterrows():
    #print(i)
    if i == 12:
        j=i
        y_icustayid.append(row["icustayid"])
        y_sex.append(row["gender"])
        y_age.append(row["age"])
        other_row = row
    else:
        #print(row["bloc"])
        if female_not_preg_set.loc[i,"icustayid"] != female_not_preg_set.loc[j,"icustayid"]:
            y_icustayid.append(row["icustayid"])
            y_sex.append(row["gender"])
            y_age.append(row["age"])
        j=i
        other_row = row

#Turn the lists into a dataframe
age_sex_notpreg_sep_df = pd.DataFrame(columns = ["icustayid", "age", "sex"])
age_sex_notpreg_sep_df["icustayid"] = y_icustayid
age_sex_notpreg_sep_df["age"] = y_age
age_sex_notpreg_sep_df["sex"] = y_sex

In [31]:
#Find the age, sex and icustayid for each patient in the young_notpreg_df dataframe
y_icustayid = []
y_age=[]
y_sex=[]
for i, row in not_preg_set.iterrows():
    #print(i)
    if i == 12:
        j=i
        y_icustayid.append(row["icustayid"])
        y_sex.append(row["gender"])
        y_age.append(row["age"])
        other_row = row
    else:
        #print(row["bloc"])
        if not_preg_set.loc[i,"icustayid"] != not_preg_set.loc[j,"icustayid"]:
            y_icustayid.append(row["icustayid"])
            y_sex.append(row["gender"])
            y_age.append(row["age"])
        j=i
        other_row = row

#Turn the lists into a dataframe
all_age_sex_notpreg_sep_df = pd.DataFrame(columns = ["icustayid", "age", "sex"])
all_age_sex_notpreg_sep_df["icustayid"] = y_icustayid
all_age_sex_notpreg_sep_df["age"] = y_age
all_age_sex_notpreg_sep_df["sex"] = y_sex

In [34]:
#Create a dataframe for each pregnant sepsis patient of potential matched (age&sex) controls
list_of_df = []
for i, row in age_sex_preg_sep_df.iterrows():
    patient_df = pd.DataFrame(columns = age_sex_notpreg_sep_df.columns)
    p_age = row["age"]
    #print(p_age)
    for j, other_row in age_sex_notpreg_sep_df.iterrows():
        
        #print("sepsis = ", row["age"])
        #print("not sepsis = ", other_row["age"])
        if other_row["age"] < p_age+365.25 and other_row["age"] > p_age-365.25:
            patient_df = patient_df.append(other_row)
            #print("J = ", j)
    list_of_df.append(patient_df)

    if i/10 == i//10:
        print(i)

0
10
20
30
40
50
60
70


In [35]:
#Create a dataframe for each pregnant sepsis patient of potential matched (age) controls 
list_of_sec_df = []
for i, row in age_sex_preg_sep_df.iterrows():
    patient_df = pd.DataFrame(columns = all_age_sex_notpreg_sep_df.columns)
    p_age = row["age"]
    #print(p_age)
    for j, other_row in all_age_sex_notpreg_sep_df.iterrows():
        
        #print("sepsis = ", row["age"])
        #print("not sepsis = ", other_row["age"])
        if other_row["age"] < p_age+365.25 and other_row["age"] > p_age-365.25:
            patient_df = patient_df.append(other_row)
            #print("J = ", j)
    list_of_sec_df.append(patient_df)
    #patient_df.head()
    

    if i/10 == i//10:
        print(i)

0
10
20
30
40
50
60
70


In [38]:
#Building a control group (age&sex)
controlg_df = pd.DataFrame(columns = age_sex_notpreg_sep_df.columns)
for i in range(0,len(list_of_df)):
    group = random.sample(range(0, len(list_of_df[i]-1)), 5)
    for j in group:
        controlg_df = controlg_df.append(list_of_df[i].iloc[j])


In [39]:
#Building a control group (age)
controlg_sec_df = pd.DataFrame(columns = all_age_sex_notpreg_sep_df.columns)
for i in range(0,len(list_of_sec_df)):
    group = random.sample(range(0, len(list_of_sec_df[i]-1)), 5)
    for j in group:
        controlg_sec_df = controlg_sec_df.append(list_of_sec_df[i].iloc[j])

In [40]:
control_1_ids = controlg_df["icustayid"]
control_2_ids = controlg_sec_df["icustayid"]

control_1_set = DataFrame()
control_1_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(control_1_ids)]

control_2_set = DataFrame()
control_2_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(control_2_ids)]

In [45]:
old_df = pd.read_csv("../Created Data/70_and_older_mimiciii.csv")
old_notsep_ids = old_df["icustay_id"]-200000

In [46]:
old_set = DataFrame()
old_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(old_notsep_ids)]

In [49]:
pregnant_set.to_csv("../Matlab/pregnant_data_nn.csv", index=False)

In [50]:
control_1_set.to_csv("../Matlab/control_1_data_nn.csv", index=False)

In [51]:
control_2_set.to_csv("../Matlab/control_2_data_nn.csv", index=False)

In [52]:
old_set.to_csv("../Matlab/old_data_nn.csv", index=False)